# Create final spreadsheet

In [1]:
import pandas as pd

from jobsfinder.gpts import has_sales_roles
from jobsfinder.core import limit_parallel, cost_so_far

In [2]:
df = pd.read_csv("../data/04_jobs.csv")

In [3]:
df.status.value_counts()

status
No jobs              1513
Job list              161
Job open apply        110
Max depth reached      18
No content             10
Name: count, dtype: int64

In [4]:
hiring = df[df.status == "Job list"].copy()

In [5]:
len(hiring)

161

In [6]:
hiring.head(2)

,CompanyName,Website,scrape_status,md_status,md,html_length,md_length,valid_website,history,status,error,jobs
9,Remofirst,https://www.remofirst.com/,Success,Success,Global HR Solutions for Remote Teams \| Remofi...,1638294,10079,valid,"[""https://www.remofirst.com/"", ""https://www.re...",Job list,NaN,"[""Executive Assistant"", ""Customer Success Spec..."
13,Adopt-Ag.com,https://www.adopt-ag.com,Success,Success,Agritech \| Adopt\n\n\ntop of page[![New Black...,712822,2764,valid,"[""https://www.adopt-ag.com"", ""https://www.adop...",Job list,NaN,"[""Full Stack Software Engineer"", ""Data Engineer""]"


In [7]:
await has_sales_roles(hiring.jobs.iloc[0])

SalesRoles(qualified=True, best_roles=['Account Executive', 'Director of Customer Success'], email_line="I saw you're hiring for an account executive and a director of customer success.")

In [9]:
tasks = [has_sales_roles(x) for x in hiring.jobs]

quals = await limit_parallel(tasks)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161/161 [00:35<00:00,  4.52it/s]


In [10]:
hiring['qualified'] = [x.qualified for x in quals]
hiring['best_roles'] = [",".join(x.best_roles) for x in quals]
hiring['line'] = [x.email_line for x in quals]

In [11]:
hiring.qualified.value_counts()

qualified
False    92
True     69
Name: count, dtype: int64

In [12]:
qualified = hiring[hiring.qualified].copy()

In [13]:
len(qualified)

69

In [14]:
qualified.Website

9         https://www.remofirst.com/
82           https://www.rutter.com/
91           https://www.alltius.ai/
100         https://thatchhealth.com
105              http://mintlify.com
                    ...             
1779       https://www.sixstring.us/
1861            http://zetachain.com
1864          https://sievedata.com/
1873         https://www.zootools.co
1965    https://www.trydownstream.io
Name: Website, Length: 69, dtype: object

checking manually, 2 (precision) errors in the whole flow

In [15]:
def _create_email(x):
    return f"""
Hey [firstName],

{x} Businesss must be going good!

Given your trajectory, I assume your doing everything to increase conversion. That's also the reason we've built UserGems. You can automatically capture buying signals from your prospect & put your pipeline on auto-pilot.

Let me know if that's of interest.

Best,
[Your Name]
    """.strip()

In [16]:
qualified['email'] = qualified.line.apply(_create_email)

In [17]:
qualified.to_csv("../data/06_qualifid.csv", index=False)

# Total cost for the day

In [18]:
cost_so_far()

4.905710024999999